transfer learning with keras

In [1]:
%matplotlib inline
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#释放显存
os.system("nvidia-smi | grep vpython3 | awk '{print $3}' | xargs kill -9")

/usr/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


0

In [2]:
height=224
train_batch_size=64
test_batch_size=64
FLAG = None
#assert

In [3]:
train_data_dir='/home/naruto/PycharmProjects/knifey_spoony_demo/data/knifey-spoony/train_data_link/'
test_data_dir='/home/naruto/PycharmProjects/knifey_spoony_demo/data/knifey-spoony/test/'
gen = ImageDataGenerator()
generator_train = gen.flow_from_directory(train_data_dir, target_size=(height, height),batch_size=train_batch_size)
generator_test = gen.flow_from_directory(test_data_dir, target_size=(height, height),batch_size=test_batch_size)

Found 4170 images belonging to 3 classes.
Found 530 images belonging to 3 classes.


In [4]:
# base_model = VGG16(include_top=True, weights='imagenet')需要下载，速度慢，
# include_top=true时下载也比较慢，需要测试等于true时的模型结构
# input_tensor=Input((height, height, 3))报错ValueError: The shape of the input to "Flatten" is not fully defined (got (None, None, 2048).
#  Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.，是因为此时用的Ｍｏｄｅｌ需要预先指定ｉｎｐｕｔ
base_model = ResNet50(input_tensor=Input((height, height, 3)),weights='imagenet', include_top=False)
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [5]:
for i ,layer in enumerate(base_model.layers):
    print("layer_{0}:\t{1}\t{2}".format(i,layer.trainable, layer.name))

layer_0:	False	input_1
layer_1:	True	conv1
layer_2:	True	bn_conv1
layer_3:	True	activation_1
layer_4:	True	max_pooling2d_1
layer_5:	True	res2a_branch2a
layer_6:	True	bn2a_branch2a
layer_7:	True	activation_2
layer_8:	True	res2a_branch2b
layer_9:	True	bn2a_branch2b
layer_10:	True	activation_3
layer_11:	True	res2a_branch2c
layer_12:	True	res2a_branch1
layer_13:	True	bn2a_branch2c
layer_14:	True	bn2a_branch1
layer_15:	True	add_1
layer_16:	True	activation_4
layer_17:	True	res2b_branch2a
layer_18:	True	bn2b_branch2a
layer_19:	True	activation_5
layer_20:	True	res2b_branch2b
layer_21:	True	bn2b_branch2b
layer_22:	True	activation_6
layer_23:	True	res2b_branch2c
layer_24:	True	bn2b_branch2c
layer_25:	True	add_2
layer_26:	True	activation_7
layer_27:	True	res2c_branch2a
layer_28:	True	bn2c_branch2a
layer_29:	True	activation_8
layer_30:	True	res2c_branch2b
layer_31:	True	bn2c_branch2b
layer_32:	True	activation_9
layer_33:	True	res2c_branch2c
layer_34:	True	bn2c_branch2c
layer_35:	True	add_3
layer_3

no fine-truning

In [6]:
#这个地方比较坑，调试比较费力，按照网上的操作,base_model.output的输出是tensor，以后都是tensor， 传递给其他层，切记，
# TypeError: 'NoneType' object is not callable
# 网上还将func 模型传给sequential（add）会出问题，#ValueError: Variable bn_conv1/moving_mean/biased already exists, disallowed.
# Did you mean to set reuse=True in VarScope? Originally defined at:应该是版本问题，报错说变量存在，设置需要重用
x=Flatten()(base_model.layers[172].output)#不必指定Input,或者base_model的output，就是最后一层
# transfer_layer = model.get_layer('activation_48')
# #获取权重tensor 切记和layer的区别,否则weights维度为none
#【7,7,512】直接FC到3也不错，单独只polling操作也可
# x=GlobalAveragePooling2D()(base_model.layers[172].output)
x = Dropout(0.25)(x)
x = Dense(3, activation='softmax')(x)
model = Model(base_model.input, x)

In [7]:
# ＃这里不是ｆａｌｓｅ那么就是整个网络重新训练易导致ＯＯＭ
for layer in base_model.layers:
    layer.trainable = False
#
# # compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
history_ft = model.fit_generator(
    generator_train,#可自定义
    # samples_per_epoch=4170,  # nb_train_samples，Basically steps_per_epoch = samples_per_epoch/batch_size
    # steps_per_epoch=10,  # nb_train_samples#每轮epoch遍历的samples
    validation_data=generator_test,#可自定义
    nb_epoch=10,
    verbose=1,#控制显示方式，冗长
    validation_steps=530//64,
    workers=8,
    # use_multiprocessing=True,
    # epochs=100
    # nb_val_samples=530 # nb_val_samples`->`validation_steps
)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, epochs=10, validation_data=<keras.pre..., workers=8, validation_steps=8)`
  if __name__ == '__main__':


Epoch 1/10
66/66 [==============================] - 57s 857ms/step - loss: 5.3328 - acc: 0.6400 - val_loss: 6.5979 - val_acc: 0.5176
Epoch 2/10
66/66 [==============================] - 54s 812ms/step - loss: 3.6860 - acc: 0.7579 - val_loss: 3.4095 - val_acc: 0.7039
Epoch 3/10
66/66 [==============================] - 53s 802ms/step - loss: 0.7251 - acc: 0.9324 - val_loss: 4.5239 - val_acc: 0.6652
Epoch 4/10
66/66 [==============================] - 53s 802ms/step - loss: 0.3257 - acc: 0.9713 - val_loss: 5.5997 - val_acc: 0.6094
Epoch 5/10
66/66 [==============================] - 54s 812ms/step - loss: 0.2487 - acc: 0.9792 - val_loss: 5.6345 - val_acc: 0.6030
Epoch 6/10
66/66 [==============================] - 53s 810ms/step - loss: 0.1166 - acc: 0.9881 - val_loss: 5.4358 - val_acc: 0.6330
Epoch 7/10
66/66 [==============================] - 53s 809ms/step - loss: 0.1335 - acc: 0.9884 - val_loss: 4.9898 - val_acc: 0.6330
Epoch 8/10
66/66 [==============================] - 53s 806ms/step - 

fine-truning

In [9]:
for layer in base_model.layers[:172]:
   layer.trainable = False
for layer in base_model.layers[172:]:
   layer.trainable = True
for i ,layer in enumerate(base_model.layers):
    print("layer_{0}:\t{1}\t{2}".format(i,layer.trainable, layer.name))

layer_0:	False	input_1
layer_1:	False	conv1
layer_2:	False	bn_conv1
layer_3:	False	activation_1
layer_4:	False	max_pooling2d_1
layer_5:	False	res2a_branch2a
layer_6:	False	bn2a_branch2a
layer_7:	False	activation_2
layer_8:	False	res2a_branch2b
layer_9:	False	bn2a_branch2b
layer_10:	False	activation_3
layer_11:	False	res2a_branch2c
layer_12:	False	res2a_branch1
layer_13:	False	bn2a_branch2c
layer_14:	False	bn2a_branch1
layer_15:	False	add_1
layer_16:	False	activation_4
layer_17:	False	res2b_branch2a
layer_18:	False	bn2b_branch2a
layer_19:	False	activation_5
layer_20:	False	res2b_branch2b
layer_21:	False	bn2b_branch2b
layer_22:	False	activation_6
layer_23:	False	res2b_branch2c
layer_24:	False	bn2b_branch2c
layer_25:	False	add_2
layer_26:	False	activation_7
layer_27:	False	res2c_branch2a
layer_28:	False	bn2c_branch2a
layer_29:	False	activation_8
layer_30:	False	res2c_branch2b
layer_31:	False	bn2c_branch2b
layer_32:	False	activation_9
layer_33:	False	res2c_branch2c
layer_34:	False	bn2c_bra

In [10]:
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 节省内存，类似tf的queue
history_ft = model.fit_generator(
    generator_train,#可自定义
    # samples_per_epoch=4170,  # nb_train_samples，Basically steps_per_epoch = samples_per_epoch/batch_size
    # steps_per_epoch=10,  # nb_train_samples#每轮epoch遍历的samples
    validation_data=generator_test,#可自定义
    nb_epoch=10,
    verbose=1,#控制显示方式，冗长
    validation_steps=530//64,
    workers=8,
    # use_multiprocessing=True,
    # epochs=100
    # nb_val_samples=530 # nb_val_samples`->`validation_steps
)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, epochs=10, validation_data=<keras.pre..., workers=8, validation_steps=8)`
  


Epoch 1/10
66/66 [==============================] - 54s 826ms/step - loss: 0.0196 - acc: 0.9981 - val_loss: 4.1522 - val_acc: 0.7044
Epoch 2/10
66/66 [==============================] - 53s 810ms/step - loss: 0.0160 - acc: 0.9979 - val_loss: 4.5040 - val_acc: 0.6853
Epoch 3/10
66/66 [==============================] - 53s 806ms/step - loss: 0.0299 - acc: 0.9974 - val_loss: 3.7857 - val_acc: 0.7268
Epoch 4/10
66/66 [==============================] - 53s 810ms/step - loss: 0.0220 - acc: 0.9983 - val_loss: 4.1234 - val_acc: 0.6967
Epoch 5/10
66/66 [==============================] - 53s 808ms/step - loss: 0.0109 - acc: 0.9992 - val_loss: 3.5250 - val_acc: 0.7468
Epoch 6/10
66/66 [==============================] - 53s 810ms/step - loss: 0.0101 - acc: 0.9994 - val_loss: 3.5465 - val_acc: 0.7525
Epoch 7/10
66/66 [==============================] - 53s 810ms/step - loss: 0.0110 - acc: 0.9991 - val_loss: 3.8087 - val_acc: 0.7253
Epoch 8/10
66/66 [==============================] - 53s 808ms/step - 

这里直接将特征层ｆｌａｔｔｅｎ　然后ｄｅｎｓｅ，效果明显好于前者